<a href="https://colab.research.google.com/github/mangoseeds/log/blob/main/HashTag_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
diary = pd.read_csv('/content/drive/MyDrive/diary.csv')

In [21]:
diary.head()

,일기,요약,해시태그
0,오늘은 오랜 기간 준비해온 시험을 보러 다녀왔어. 시험 전날부터 긴장과 불안이 가시...,오랜 기간 준비해온 시험을 보러 다녀왔어요. 불안과 긴장 속에서도 최선을 다해 시험...,#시험 #노력 #성취감 #안도감 #자부심
1,오늘은 오랜 기간 준비해온 시험을 보러 다녀왔어. 시험 전날부터 긴장과 불안이 가시...,오랜 기간 준비해온 시험을 보러 다녀왔어요. 시험 중에는 최대한 집중하고 최선을 다...,#시험 #노력 #집중 #결과기다림 #휴식
2,오늘은 오랜 기간 준비해온 시험을 보러 다녀왔어. 시험 전날부터 긴장과 불안이 가시...,오랜 기간 준비한 시험을 보러 다녀왔어요. 시험 중에는 최선을 다해 문제를 해결하려...,#시험 #노력 #문제해결 #결과기다림 #휴식
3,오늘은 내가 사랑하는 사람과 소중한 시간을 보내러 다녀왔어. 그 동안 서로 바쁘게 ...,사랑하는 사람과 함께한 소중한 시간이었어요. 서로의 이야기를 나누고 응원해주며 사랑...,#사랑하는사람과함께 #소중한시간 #서로의이해 #사랑과애정 #감사함
4,오늘은 내가 사랑하는 사람과 소중한 시간을 보내러 다녀왔어. 그 동안 서로 바쁘게 ...,사랑하는 사람과 함께한 소중한 시간이었어요. 서로의 이야기를 나누고 사랑을 나눌 수...,#사랑하는사람과함께 #소중한시간 #서로의이야기 #배려와사랑 #행복함


## HashTag Modelling

In [23]:
!pip install datasets transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [24]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)
from tokenizers import Tokenizer
from typing import Dict, List, Optional
from torch.utils.data import Dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display
from typing import Dict

In [25]:
model_name = "gogamza/kobart-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [36]:
# max input
max(diary['요약'].map(lambda x: len(x)).to_list())

181

In [56]:
# max output
max(diary['해시태그'].map(lambda x: len(x)).to_list())

43

In [62]:
class HashTagMakerDataset(Dataset):
  def __init__(self,
               df: pd.DataFrame,
               tokenizer: Tokenizer
               ):
    self.df = df
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    row = self.df.iloc[index, :]
    text1 = row[1]
    text2 = row[2]

    encoder_text = text1
    decoder_text = text2
    model_inputs = self.tokenizer(encoder_text, max_length=181, truncation=True)

    with self.tokenizer.as_target_tokenizer():
      labels = tokenizer(decoder_text, max_length=43, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    del model_inputs['token_type_ids']

    return model_inputs

In [61]:
# Train, Test split - 현재는 이 과정 skip
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(diary, test_size=0.1, random_state=42) # train, test 분리
print(len(df_train), len(df_test))

194 22


In [63]:
train_dataset = HashTagMakerDataset(df_train, tokenizer)
test_dataset = HashTagMakerDataset(df_test, tokenizer)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = model)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

In [64]:
model_path = "/content/drive/MyDrive/졸업프로젝트/"

training_args = Seq2SeqTrainingArguments(
    output_dir=model_path,
    overwrite_output_dir=True,
    num_train_epochs=24,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_steps=500,
    save_steps=1000,
    warmup_steps=300,
    prediction_loss_only=True,
    evaluation_strategy="steps",
    save_total_limit=3
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [65]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=312, training_loss=2.0116772773938303, metrics={'train_runtime': 3038.3985, 'train_samples_per_second': 1.532, 'train_steps_per_second': 0.103, 'total_flos': 132796313395200.0, 'train_loss': 2.0116772773938303, 'epoch': 24.0})

In [66]:
trainer.save_model("/content/drive/MyDrive/졸업프로젝트/")

In [97]:
evaluation_results = trainer.evaluate(eval_dataset=test_dataset)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [99]:
evaluation_results

{'eval_loss': 1.5491961240768433,
 'eval_runtime': 7.4702,
 'eval_samples_per_second': 2.945,
 'eval_steps_per_second': 0.268,
 'epoch': 24.0}

In [98]:
loss = evaluation_results["eval_loss"]
print("평가 손실:", loss)

평가 손실: 1.5491961240768433


In [82]:
# 테스트할 문장
test_sentence = "명절을 맞아 고향에 돌아와 가족과 함께 보낸 특별하고 따뜻한 하루였다. 가족의 사랑과 고향의 소중함을 깨달았다."

# 입력 문장을 토큰화하여 인코딩
input_ids = tokenizer.encode(test_sentence, return_tensors="pt")

# 모델에 입력 전달하여 디코딩
output = model.generate(input_ids)

# 디코딩된 출력을 토크나이저를 사용하여 텍스트로 변환
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

# 결과 출력
print("입력 문장:", test_sentence)
print("모델 출력:", decoded_output)

입력 문장: 명절을 맞아 고향에 돌아와 가족과 함께 보낸 특별하고 따뜻한 하루였다. 가족의 사랑과 고향의 소중함을 깨달았다.
모델 출력: #명절 #돌아와 가족과함께 #특별한 하루 #가족과함께 #행복


In [93]:
def make_tag(text):
  # 입력 문장을 토큰화하여 인코딩
  input_ids = tokenizer.encode(text, return_tensors="pt")

  # 모델에 입력 전달하여 디코딩
  output = model.generate(input_ids)

  # 디코딩된 출력을 토크나이저를 사용하여 텍스트로 변환
  decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

  # 결과 출력
  print("입력 문장:", text)
  print("모델 출력:", decoded_output)

In [94]:
make_tag(diary.iloc[-2,1])

입력 문장: 반려동물과 함께 특별한 일상을 기록하며 즐거움과 행복을 느낀 특별한 경험을 한 날이었어.
모델 출력: #강아지 #즐거움과행복 #충실한친구 #사랑스러운모습 #


In [95]:
make_tag(diary.iloc[-10,1])

입력 문장: 취업 준비를 하며 부족함과 노력을 돌아보고 성장의 기회를 찾은 특별한 경험을 한 날이었어.
모델 출력: #취준생 #취업준비 #부족함과노력 #성장의기회 #


In [103]:
make_tag(diary.iloc[-45,1])

입력 문장: 요가를 시작해서 몸과 마음이 편안해지고 건강을 유지하고 있어요.
모델 출력: #요가 #건강 #편안함 #유연성 #숨쉬기 #
